In [77]:
import pandas as pd
import numpy as np
from pathlib import Path
import subprocess



raw = Path('/home/is153802/neurospin/acquisition/neuromag/data/petit_prince')
scan_path = Path('/home/is153802/neurospin/acquisition/database/Prisma_fit')
output_folder = Path("/home/is153802/workspace_LPP/data/MEG/LPP/anat")

subj = [str(sub) for sub in raw.iterdir()]
list_scans = []
with open('/home/is153802/code/source-reconstruction/anat_mri.txt', 'r') as fh:
    for line in fh:
        list_scans.append((line))

list_scans = [scan.replace('\n','') for scan in list_scans]

list_scans_subjects = [scan[11:19] for scan in list_scans]

# To generate before the txt file with the anat MRI:
# cd /neurospin/acquisition/database/Prisma_fit 
# find -maxdepth 3 -name '*mprage*' -type d > /home/is153802/code/source-reconstruction/anat_mri.txt
# chmod 777 /home/is153802/code/source-reconstruction/anat_mri.txt


In [78]:
subjects = []
for sub in subj: 
    index = sub.rfind("petit_prince/")
    subjects.append(sub[index+13:])
subjects.remove('rt220104_rt220104')
subjects.remove('ml100438_ml100438')
subjects.remove('empty_room')
subjects.append('ml_100438')
subjects.append('rt_220104')
# TODO figure out the ab73???
subjects = [subject.replace('_','') for subject in subjects]

In [79]:
list_scans_final = [scan for i, scan in enumerate(list_scans) if list_scans_subjects[i] in (subjects)]

In [80]:
# For each subject:
# Get their subset of scans
# Sort it by date, and take the latest
scan_to_move = []

for sub in subjects:
    list_scan_specific = [scan for i, scan in enumerate(list_scans_final) if scan[11:19] == sub]
    # If no scan for the subject, skip it
    if list_scan_specific == []:
        continue
    # print(f'Sub {sub} ! list {list_scan_specific}')
    list_scan_specific.sort(reverse = True)


    scan_to_move.append((list_scan_specific[0]))
scan_to_move = [scan.replace('./','') for scan in scan_to_move]

In [83]:
for scan in scan_to_move:
    scan_path_specific = scan_path / scan

    sub = scan[9:17]
    bashCommand = f"mkdir -p {output_folder} "
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    bashCommand = f"cp -r   {scan_path_specific} {output_folder/sub}"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()